<a href="https://colab.research.google.com/github/sandrazivanovska/sandrazivanovska.github.io/blob/main/Lab1_1_vnp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab Exercise 1. Scraping Static Websites


This is the warmup task for the first laboratory exercise. It consists of scraping static Websites with BeautifulSoap.

 It should be completed at home and presented at the laboratory.

**Total points: 2**

### Task Description

Scrape the information about the products on the following page:
https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/

For each product scrape:


*   Product title (selector `'.wd-entities-title'`)
*   Product regular price (selector `'.woocommerce-Price-amount'`)
*   Product discount price (if available), same selector as regular price
*   URL to the product page
*   Add to cart button URL

***Help: There are multiple product pages, for each page you need to send a separate request***


Save the results as a DataFrame object

You can add as many code cells as you need.

________________________________________________________________

### Requirements

Import libraries and modules that you are going to use

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### Send HTTP request to the target Website

In [2]:
url = 'https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/'
response = requests.get(url)

check the response status code

In [4]:
status_code = response.status_code
status_code

200

### Parse the response content with BeautifulSoap

In [5]:
raw_html = response.text
soup = BeautifulSoup(raw_html, 'html.parser')

### Extract data from the BeautifulSoap object using any selectors, attribute identifiers, etc.

* Product title (selector '.wd-entities-title')
* Product regular price (selector '.woocommerce-Price-amount')
* Product discount price (if available), same selector as regular price
* URL to the product page
* Add to cart button URL

Repeat the extraction process for each page of products

In [8]:
def extraxt_products_characteristics(product):
    name = product.select_one('.wd-entities-title').text.strip()
    prices = product.select('.woocommerce-Price-amount')
    regular_price = prices[0].text.strip() if prices else None
    discount_price = prices[1].text.strip() if len(prices) > 1 else None
    product_url = product.select_one('.wd-entities-title a')['href']
    add_to_cart_button = product.select_one('.add_to_cart_button')['href'] if product.select_one('.add_to_cart_button') else None

    product_list = {
        'title': name,
        'regular_price': regular_price,
        'discount_price': discount_price,
        'product_url': product_url,
        'add_to_cart_button': add_to_cart_button
    }

    return product_list

In [9]:
all_products = []
base_url = "https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/"
page_num = 1

while True:
    if page_num == 1:
        url = base_url
    else:
        url = f"{base_url}page/{page_num}/"  # Поправен URL без дополнителна коса црта

    response = requests.get(url)

    print(f"Обработувам страница {page_num}: {url}")

    if response.status_code != 200:
        print("Не може да се пристапи до страницата:", url)
        break

    soup = BeautifulSoup(response.text, "html.parser")
    product_elements = soup.select('.product-wrapper')  # Обезбедете дека класата е точна

    print("Наодени производи:", len(product_elements))

    if not product_elements:
        print("Нема повеќе производи за процесирање.")
        break

    for product in product_elements:
        result = extraxt_products_characteristics(product)
        all_products.append(result)

    page_num += 1


Обработувам страница 1: https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/
Наодени производи: 24
Обработувам страница 2: https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/2/
Наодени производи: 24
Обработувам страница 3: https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/3/
Наодени производи: 24
Обработувам страница 4: https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/4/
Наодени производи: 24
Обработувам страница 5: https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/5/
Наодени производи: 24
Обработувам страница 6: https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/6/
Наодени производи: 24
Обработувам страница 7: https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/7/
Наодени производи: 24
Обработувам страница 8: https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/8/
Наодени производи: 24
Обработувам страница 9: https://clevershop.mk/product-c

### Create a pandas DataFrame with the scraped products

In [13]:
all_products_dataFrame = pd.DataFrame(all_products)
all_products_dataFrame

,title,regular_price,discount_price,product_url,add_to_cart_button
0,Acer A315-23-A7KD,17.590 ден,None,https://clevershop.mk/product/acer-a315-23-a7kd/,?add-to-cart=21494
1,Acer A315-23-R5P2,27.490 ден,None,https://clevershop.mk/product/acer-a315-23-r5p2/,?add-to-cart=21510
2,ACER Aspire 1 A115-22,18.999 ден,15.999 ден,https://clevershop.mk/product/acer-aspire-1-nx...,?add-to-cart=20826
3,Acer Aspire 3 A315-23-R26A,29.990 ден,None,https://clevershop.mk/product/acer-aspire-3-a3...,?add-to-cart=21516
4,Acer Aspire 3 A315-58-33WK,24.490 ден,None,https://clevershop.mk/product/21498/,?add-to-cart=21498
...,...,...,...,...,...
315,Monitor 27 Philips 272E1GAJ/00 VA 1ms 144Hz,12.890 ден,None,https://clevershop.mk/product/monitor-27-phili...,?add-to-cart=12618
316,Philips 24″ 243V7QDSB,8.390 ден,None,https://clevershop.mk/product/philips-24%e2%80...,?add-to-cart=12396
317,Philips 27″ 278E1A/00 4K UHD IPS,18.990 ден,None,https://clevershop.mk/product/hp-27%e2%80%b3-2...,?add-to-cart=12218
318,Philips 279C9-00 MON LED 27″ 3840 x 2160 5Ms 6...,26.990 ден,None,https://clevershop.mk/product/philips-279c9-00...,?add-to-cart=12578


Save the dataframe as `.csv`

In [14]:
all_products_dataFrame.to_csv('mobilni-laptopi-i-tableti.csv', index = False)